# Board Connection Status Dashboard

This dashboard provides a real-time view of board connection statuses using ipywidgets and plotly.

In [102]:
# Install required packages
!pip install ipywidgets voila pandas plotly psycopg2-binary

In [103]:
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import psycopg2
from IPython.display import display, clear_output
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
import os
from pathlib import Path

# Configure logging
import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Load environment variables from .env file
dotenv_path = Path('../../.env')
load_dotenv(dotenv_path)

False

In [104]:
import os
import logging
from sqlalchemy import create_engine, text

# Database configuration from container environment variables with fallbacks
DB_CONFIG = {
    'host': os.getenv('POSTGRES_HOST', 'timescaledb'),  # Use container service name
    'port': int(os.getenv('POSTGRES_PORT', 5432)),
    'database': os.getenv('POSTGRES_DB', 'perocube'),
    'user': os.getenv('POSTGRES_USER', 'postgres'),
    'password': os.getenv('POSTGRES_PASSWORD', 'postgres')
}

# Print database connection info (excluding password)
print(f"Database connection: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']} as {DB_CONFIG['user']}")

# Database connection function
def get_db_connection():
    """
    Create a SQLAlchemy database engine from configuration.
    
    Args:
        config: Dictionary containing database connection parameters
        
    Returns:
        SQLAlchemy engine instance
    """
    try:
        connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        # Store connection string as attribute of engine for external access
        engine = create_engine(connection_string)
        engine.connection_string = connection_string  # This makes it accessible via engine.connection_string
        
        # Test the connection
        with engine.connect() as conn:
            result = conn.execute(text("SELECT 1"))
            logging.info(f"Database connection successful: {DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}")
        return engine
    except Exception as e:
        logging.error(f"Database connection failed: {str(e)}")
        raise

Database connection: timescaledb:5432/perocube as postgres


In [105]:
# Function to fetch board status data
def fetch_board_status(check_time=None):
    if check_time is None:
        check_time = datetime.now()
        
    try:
        engine = get_db_connection()
        # Use %s style parameter for PostgreSQL
        query = "SELECT * FROM get_board_connection_status(%s)"
        df = pd.read_sql_query(query, engine, params=(check_time,))
        return df
    except Exception as e:
        logging.error(f"Error fetching board status: {str(e)}")
        raise

In [106]:
# Create widgets
date_picker = widgets.DatePicker(
    description='Date:',
    value=datetime.now().date()
)

time_picker = widgets.Text(
    description='Time (HH:MM):',
    value=datetime.now().strftime('%H:%M')
)

refresh_button = widgets.Button(
    description='Refresh Data',
    icon='refresh'
)

auto_refresh = widgets.Checkbox(
    value=False,
    description='Auto-refresh (30s)',
    indent=False
)

status_output = widgets.Output()

In [107]:
# Function removed - using grid view only

In [108]:
def create_status_grid(df):
    # Create a matrix for all possible board-channel combinations
    boards = range(1, 11)  # 10 boards
    channels = range(1, 25)  # 24 channels
    
    # Create the base figure
    fig = go.Figure()
    
    # Create a DataFrame with all possible combinations
    grid_data = pd.DataFrame([(b, c) for b in boards for c in channels],
                           columns=['board_number', 'channel_number'])
    
    # Merge with actual data
    grid_data = grid_data.merge(df, on=['board_number', 'channel_number'], how='left')
    
    # Define better colors for visibility (less saturated)
    active_color = '#A5D6A7'  # Pale green
    inactive_color = "#FF4040"  # Pale red
    unused_color = "#F5F5F5"  # Very light gray
    
    # Create text matrix (transposed: channels x boards)
    text_matrix = []
    colors_matrix = []
    hover_matrix = []
    
    for channel in channels:  # Outer loop is now channels
        text_row = []
        colors_row = []
        hover_row = []
        channel_data = grid_data[grid_data['channel_number'] == channel]
        
        for board in boards:  # Inner loop is now boards
            cell_data = channel_data[channel_data['board_number'] == board].iloc[0] if len(channel_data[channel_data['board_number'] == board]) > 0 else pd.Series()
            # Create cell text and hover text
            if pd.isna(cell_data.get('solar_cell_name')):
                text_row.append('')  # Empty text for unused cells
                hover_row.append('Channel not in use')
                colors_row.append(unused_color)
            else:
                # Show cell name and pixel identifier
                cell_name = cell_data['solar_cell_name']
                pixel_id = cell_data['pixel_identifier']
                text_row.append(f"{cell_name}\n{pixel_id}") # Display cell name and pixel ID
                hover_row.append(f"<b>Board {board}, Channel {channel}</b><br>" + 
                               f"Solar Cell: {cell_name}<br>" +
                               f"Pixel ID: {pixel_id}<br>" +
                               f"<b>Status: {'Active' if cell_data['is_active'] == 1 else 'Inactive'}</b>")
                colors_row.append(active_color if cell_data['is_active'] == 1 else inactive_color)
        
        text_matrix.append(text_row)
        colors_matrix.append(colors_row)
        hover_matrix.append(hover_row)
    
    # Create heatmap with custom text
    fig.add_trace(go.Heatmap(
        z=[[1 for _ in boards] for _ in channels],  # Dummy z values
        text=text_matrix,
        texttemplate='%{text}',
        textfont={"size": 10, "color": "black", "family": "Arial"}, # Adjusted font size slightly
        colorscale=[[0, 'rgba(0,0,0,0)'], [1, 'rgba(0,0,0,0)']],
        showscale=False,
        x=list(boards),
        y=list(channels),
        customdata=hover_matrix,
        hovertemplate='%{customdata}<extra></extra>',
        hoverlabel=dict(
            bgcolor='white',
            font_size=13,
            font_family="Arial",
            bordercolor='black',
            align="left"
        )
    ))
    
    # Update layout
    fig.update_layout(
        title=dict(
            text='Board Connection Status Grid',
            x=0.5,
            font=dict(size=24, family="Arial Black")
        ),
        xaxis_title=dict(
            text='Board Number',
            font=dict(size=14, family="Arial")
        ),
        yaxis_title=dict(
            text='Channel Number',
            font=dict(size=14, family="Arial")
        ),
        height=1100,  # Taller to give more room for text
        width=1200,  # Adjusted width for wider cells
        margin=dict(l=80, r=80, t=100, b=60),  # Adjusted margins (removed legend space, added top for xaxis)
        paper_bgcolor='white',
        plot_bgcolor='white',
        xaxis=dict(
            tickmode='linear',
            tick0=1,
            dtick=1,
            tickangle=0,
            constrain='domain',
            showgrid=False,  # Remove x-axis grid lines
            gridcolor='#CCCCCC',
            linecolor='black',
            mirror=True,
            tickfont=dict(size=12),
            title_standoff=15, # Adjusted for top position
            side='top' # Move x-axis to top
        ),
        yaxis=dict(
            tickmode='linear',
            tick0=1,
            dtick=1,
            autorange='reversed',  # Keep reversed to have channel 1 at top
            constrain='domain',
            scaleanchor='x',
            scaleratio=0.5,  # Make cells wider
            tickfont=dict(size=12),
            title_standoff=20,
            showgrid=False,  # Remove y-axis grid lines
            gridcolor='#E5E5E5',
            linecolor='black',
            mirror=True
        )
        # Legend related shapes and annotations are removed
    )
    
    # Add cell colors using shapes
    for i, channel in enumerate(channels):
        for j, board in enumerate(boards):
            fig.add_shape(
                type="rect",
                x0=board-0.5, x1=board+0.5,
                y0=channel-0.5, y1=channel+0.5,
                fillcolor=colors_matrix[i][j],
                line=dict(width=1, color="white"),
                layer="below"
            )
    
    return fig

In [109]:
# Update function
def update_display(b=None):
    with status_output:
        clear_output(wait=True)
        try:
            # Combine date and time
            date = date_picker.value
            time = datetime.strptime(time_picker.value, '%H:%M').time()
            check_time = datetime.combine(date, time)
            
            # Fetch and display data
            df = fetch_board_status(check_time)
            fig = create_status_table(df)
            fig.show()
            
            # Display summary statistics
            total_boards = len(df)
            active_boards = df['is_active'].sum()
            print(f"Total Boards: {total_boards}")
            print(f"Active Boards: {active_boards}")
            print(f"Inactive Boards: {total_boards - active_boards}")
            
        except Exception as e:
            print(f"Error: {str(e)}")

refresh_button.on_click(update_display)

In [110]:
# Auto-refresh handler
def auto_refresh_handler(change):
    if change['new']:  # If auto-refresh is enabled
        update_display()
        # Schedule next update in 30 seconds
        from IPython.display import Javascript
        display(Javascript("""
        setTimeout(function() {
            if (document.querySelector('#auto-refresh').checked) {
                document.querySelector('#refresh-button').click();
            }
        }, 30000);
        """))

auto_refresh.observe(auto_refresh_handler, names='value')

# Update function
def update_display(b=None):
    with status_output:
        clear_output(wait=True)
        try:
            # Combine date and time
            date = date_picker.value
            time = datetime.strptime(time_picker.value, '%H:%M').time()
            check_time = datetime.combine(date, time)
            
            # Fetch data and create visualization
            df = fetch_board_status(check_time)
            fig = create_status_grid(df)
            fig.show()
            
            # Display summary statistics
            total_boards = len(df)
            active_boards = df['is_active'].sum()
            print(f"Total Connections: {total_boards}")
            print(f"Active Connections: {active_boards}")
            print(f"Inactive Connections: {total_boards - active_boards}")
            
        except Exception as e:
            print(f"Error: {str(e)}")

refresh_button.on_click(update_display)

In [111]:
# Layout
controls = widgets.HBox([date_picker, time_picker, refresh_button, auto_refresh])
dashboard = widgets.VBox([controls, status_output])
display(dashboard)

# Initial update
update_display()